In [ ]:
import cv2
import math
import numpy as np
import os
import matplotlib.pyplot as plt
from timeit import default_timer as time 
import tkinter as tk 
import datetime
from tkinter import filedialog

# Simple Downsampling

In [ ]:
def simple_downsmpl(img ,srf):
  
    im_array = np.array(img)
    im_downsampled = im_array[::srf, ::srf, :]
    return im_downsampled

# Simple Upsampling --not working

In [ ]:
def simple_upsmpl(img,srf):
    
    height, width ,col_ch = img.shape
    upsampled_image = np.zeros((height * 2, width * 2,col_ch)).astype(int)
    upsampled_image[::srf, ::srf,:] = img
    return upsampled_image

# Floating window for video input

In [ ]:
def show_file_location(event):
    global file_path
    file_path = filedialog.askopenfilename()

root = tk.Tk()
root.geometry("250x150")

label = tk.Label(root, text="Select a file by \n clicking on a floating window \n \n NOTE --> Close the window after selection")
label.pack(pady=35)


label.bind("<Button-1>", show_file_location)

root.mainloop()

# Dark Channel Prior  -check pt1

In [ ]:
def DCP(img,window):                                    
    #Spliting image into inc=dividual color components--> 
    R=img[:,:,2]
    G=img[:,:,1]
    B=img[:,:,0]
    h,w,c = img.shape
    
    #Flatening of image for loop jamming(reducing the number of loops) [other wise have to use 2 nested loop instead of 1]
    R1 = np.ravel(R1)
    G1 = np.ravel(G1)
    B1 = np.ravel(B1)
    min_col_ch_img = np.empty(R1.size)    #[check pt1] #may be R1.size = R.size (check)
    
    for i in range(0,R.size): 
        min_col_ch_img[i] = min(B1[i],G1[i],R1[i])
    
    #Reshaping back to 2D image--> 
    min_col_ch_img = min_col_ch_img.reshape(h,w)
 
    kernel = np.ones((window,window), dtype = int)
    dark_ch = cv2.erode(min_col_ch_img,kernel)
    return dark_ch

# Global atmospheric light estimator (A)

In [ ]:
def A_estimator(img,dark_ch):
    h,w,c = img.shape
    
    #Spliting image into individual color components--> 
    B = img[:,:,0]
    G = img[:,:,1]
    R = img[:,:,2]
    
    #Flatening of image for loop jamming(reducing the number of loops) [other wise have to use 2 nested loop instead of 1]
    B1 =B.ravel()
    G1 =G.ravel()
    R1 =R.ravel()
    
    #satcking flattened image to make 2d image in which [row 0] represents B coMponent ; [row 1] represents G component ;[row 2] represents R component 
    img1= np.vstack((B1,G1,R1))                
    dark_ch = dark_ch.ravel()

    #Descending order argsorting for 0.1% brightest pixel in dark_channel-->
    top_point_1_perc = dark_ch.argsort()[::-1]
    
    Atm_sum = np.zeros([3])
    for i in range(h*w//1000):
        Atm_sum1[0]=AL_sum1[0] +img1[0][top_point_1_perc [i]]
    for i in range(h*w//1000):
        Atm_sum1[1]=AL_sum1[1] +img1[1][top_point_1_perc [i]]
    for i in range(h*w//1000):
        Atm_sum1[2]=AL_sum1[2] +img1[2][top_point_1_perc [i]] 
        
    Atm_sum = Atm_sum.reshape([1,3])  
    A = Atm_sum/(h*w//1000)
    return A

# Estimating trans estimator using Aerial perspective

In [ ]:

def Trans_estimation(img, A, window, omega):
    #omega = 0.95
    img_temp = np.empty(img.shape, img.dtype)
    for i in range(3):
        img_temp[:,:,i] = img[:,:,i]/A[0,i]
    trans = 1 - omega*DCP(img_temp, window)
    return trans

# Guided filter

In [ ]:
def Guided_filter(img, tr,guide_area,eps):
    img_mean = cv2.boxFilter(img, cv2.CV_64F, (guide_area ,guide_area))
    tr_mean = cv2.boxFilter(tr, cv2.CV_64F, (guide_area ,guide_area))
    img_corel = cv2.boxFilter(img*img, cv2.CV_64F, (guide_area ,guide_area))
    img_t_corel = cv2.boxFilter(img*tr, cv2.CV_64F, (guide_area ,guide_area))
    
    img_var = img_corel - img_mean*img_mean
    img_t_covar = img_t_corel - img_mean*tr_mean
    
    const_a = img_t_covar / (img_var + eps)
    const_b = tr_mean - const_a*img_mean
    
    mean_a = cv2.boxFilter(const_a, cv2.CV_64F,(guide_area ,guide_area))
    mean_b = cv2.boxFilter(const_b, cv2.CV_64F, (guide_area ,guide_area))
    
    result = mean_a * img + mean_b
    
    return result

# Main dehazing function

In [ ]:
def dehaze(img, r, n ,window):    
    #threshold
    t_0 = 0.1
    omega = 0.8
    eps = 0.001
    
    normalized_img = np.float64(img)/255
    dark_J = DCP(normalized_img, window)
    A = A_estimator(normalized_img, dark_J)
    t = Trans_estimation(normalized_img, A, r, omega)
    
    img_gs = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gs = np.float64(img_gs)/255
    guide_area =40
    t_fil = Guided_filter(img_gs,t,guide_area,eps)
    
    t_thre = cv2.max(t_fil, t_0)
    J = np.empty(normalized_img.shape, normalized_img.dtype)
    for i in range(3):
        J[:,:,i] = (normalized_img[:,:,i]-A[0,i])/t_thre + A[0,i]
    
    return J

# Extracting frames from video 

In [ ]:
file_path = file_path.replace("/", "\\")

vid_obj = cv2.VideoCapture(file_path)
fps = vid_obj.get(cv2.CAP_PROP_FPS)
no_of_frame = 0
while(True):
    success, frame = vid_obj.read()
    if success==False:
        break
    no_of_frame += 1
    print('Stored frame:', no_of_frame)
    #params = []
    #params.append(1)
    cv2.imwrite("C:\\Users\\qa284\\Desktop\\major project final code settings\\@NEW BEGININIGS\\testing\\extracted_frames\\frame" + "_%d.png" % no_of_frame, frame) #, params)
vid_obj.release()
# params is optional 



# Critical parameters

In [ ]:
r=5
n=8
window = 11
srf =2 #srf is sampling rate factor

# Dehazing frames

In [ ]:
x_fc=[]
y_time=[]



for f in range(1,no_of_frame+1):
    
    im_file = os.path.join('C:\\Users\\qa284\\Desktop\\major project final code settings\\@NEW BEGININIGS\\testing\\extracted_frames', 'frame_'+str(f)+'.png')
    img = cv2.imread(im_file)
    
    #timer starts 
    start = time()
    
    img = simple_downsmpl(img,srf) 
    J = dehaze(img, r, n,window)
    
    #timer ends 
    end = time()
    
    elpsd_time =end - start
    x_fc.append(f)
    y_time.append(elpsd_time)
    
    im_file = os.path.join('C:\\Users\\qa284\\Desktop\\major project final code settings\\@NEW BEGININIGS\\testing\\dehazed_frames', 'frame_'+str(f)+'.png')
    cv2.imwrite(im_file,J*255)
    print('Dehazed frame:',f ,"\t | \t Time_taken:",y_time[f-1],"seconds")
    print("--------------------------------------------------------------------------")

# Frame assembling to make dehazed video --not working

In [ ]:
fps = vid_obj.get(cv2.CAP_PROP_FPS)
print(fps)

In [ ]:
 
fps = vid_obj.get(cv2.CAP_PROP_FPS)
demo = cv2.imread('C:\\Users\\qa284\\Desktop\\major project final code settings\\@NEW BEGININIGS\\testing\\extracted_frames\\frame_1.png')
demo = np.min(demo, axis = 2)
size = demo.shape
size = (size[1],size[0])
print('Assembling dehazed frames to make up Video ==>')
cap = cv2.VideoWriter("C:\\Users\\qa284\\Desktop\\major project final code settings\\@NEW BEGININIGS\\testing\\dehazed_video.avi",cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, size)
for i in range(1,no_of_frame+1):
    im_file = os.path.join('C:\\Users\\qa284\\Desktop\\major project final code settings\\@NEW BEGININIGS\\testing\\dehazed_frames', 'frame_'+str(i)+'.png')
    img = cv2.imread(im_file)
    cap.write(img)
cap.release()


# Timing plots and other details --not working

In [ ]:
print("----------------------------------------------------------------------------------")
frames = vid_obj.get(cv2.CAP_PROP_FRAME_COUNT)
fps = vid_obj.get(cv2.CAP_PROP_FPS)
seconds = round(frames / fps)
video_time = datetime.timedelta(seconds=seconds)
print(f"Frames per second of video--> {fps}")
print("----------------------------------------------------------------------------------")
print(f"video duration [hh:mm:ss]--> {video_time}")
print("----------------------------------------------------------------------------------")
print("Total number of frames in video:",len(res))
print("----------------------------------------------------------------------------------")
print("Total time for dehazing video:",sum(res))
print("----------------------------------------------------------------------------------")
print("Average time for dehazing per frame:",sum(res)/len(res))
print("----------------------------------------------------------------------------------")
print("maximum processing time for a frame", max(res) )
print("----------------------------------------------------------------------------------")
print("minimum processing time for a frame", min(res) )
print("----------------------------------------------------------------------------------")
# print(y_time)
# print("----------------------------------------------------------------------------------")
# print(res)
# print("----------------------------------------------------------------------------------")
# print(x_fc)
# print("----------------------------------------------------------------------------------")
# print(len(y_time))
# print("----------------------------------------------------------------------------------")
plt.scatter(x_fc,y_time)
plt.ylabel("time [seconds] -->")
plt.xlabel("Frame Number-->")
plt.text(3, 4.95 , f'frames in video: { len(res) } ' , bbox=dict(facecolor='red', alpha=0.5))
plt.show()